<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/screen_align.png" style="margin: 0 auto;">


<h1 style="text-align: center; font-size=58px;">Bulk Writes</h1>

In this lesson we're going to discuss a different type of writes called "Bulk Writes," and the performance implications of these kinds of writes.

<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/replica_set_many_writes.png" style="margin: 0 auto;">

Often times, our applications will encounter situations in which they need to perform a series of writes at once, and in some cases these writes have a causal effect on one another. One failing or succeeding in a group of operations, may affect your application logic.

In this case, a user is purchasing food items on our grocery store application, and we are updating the database to reflect the new quantities we have in stock. This person bought 2 (point) apples, so we decrement the quantity by 2.

When our application receives these writes, one option it has is to send each (point) of these writes (point) one at a time.

<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/replica_set_first_update_one.png" style="margin: 0 auto;">

So the client sends a write (point) over to the database (point).

<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/replica_set_first_update_one_ack.png" style="margin: 0 auto;">

And at some point later, the client receives an acknowledgement that the write succeeded. Nice! Now let's send over the next write.

<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/replica_set_second_update_one.png" style="margin: 0 auto;">

So we send over our next write,

<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/replica_set_second_update_one_ack.png" style="margin: 0 auto;">

And then eventually get our acknowledgement. Now we just performed two write operations, and it required two round trips to the database.

That's a round trip to the database for each write operation - but if we already knew all the writes we wanted to perform, why is our client sending them each one at a time? You probably see where this is going.

<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/replica_set_update_many_ordered.png" style="margin: 0 auto;">

So what we can do instead is "batch" these inserts together, and then send them in bulk. The exact method of grouping documents together is implemented differently in each driver, because the data structures are different, but the general idea is the same: Package a bunch of writes into a batch, usually a list or array (but again, the implementation is different in each language), and then send that whole batch to MongoDB.

This is the implementation of bulk writes in the Mongo shell, and you can copy this from the lecture notes if you want to try it out. But it will look different in your chosen programming language, so bear that in mind.

<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/replica_set_update_many_ordered_ack.png" style="margin: 0 auto;">

When a client sends a bulk write, it gets one acknowledgement back from the database for the whole batch.

This is a benefit to our application's performance, because it limits the effect of latency on the overall speed of the operation. If it takes one second for each round trip, then sending one write at a time takes four seconds. But if we can send all four writes in one round trip, then sending four writes only takes one second.

Now, the default behavior of a bulk write in Mongo is an ordered execution of these (point) writes (point). And in the ordered bulk write, any failure will stop execution of the rest of the batch. This benefits us in this case, because these (point) writes might have an effect on each other, like if two different update operations want to buy 4 sticks of butter, but there's only one stick left. In that situation, the first operation in the batch should get the last stick of butter, and the second operation should error out. That's why we need these (point) executed in order.

The bulk write would throw an error on this (point) update statement, and then return an acknowledgement to the client **before** trying to purchase anymore (point) items. The acknowledgement (point) we get back will then tell us if something errored out.

### Ordered Bulk Write

- The default setting for bulk writes in MongoDB
- Executes writes sequentially
    - Will end execution after first write failure

By default, bulk writes in Mongo will be ordered. This means that even though we send all the writes at the same time, the replica (point) set will apply them in the order they were sent.

Sending an ordered bulk write implies that each write (point to butter or eggs) in the batch depends on all the writes (point to the ones above it) that occurred before it. So if a write operation results in an error, all subsequent writes will not be executed, because Mongo assumes that those (point to ones below it) writes were expecting this (point) write to succeed.

<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/replica_set_update_many_unordered.png"/>

But there's also a chance that the writes in our batch are not dependent on each other. In this case, we've just received a shipment of food to the warehouse, and we want to update the new food quantities in stock.

Because all these (point) changes are additive, we don't need all of them to be executed in order, because they're not causally related. So I passed this "ordered: false" flag to the bulk write command, which will execute them in parallel. If some of them fail (for whatever reason), we can still continue on with the execution of other operations in the batch.

<img src="https://s3.amazonaws.com/edu-static.mongodb.com/lessons/M220/notebook_assets/replica_set_update_many_unordered_ack.png"/>

And when we receive an acknowledgement back from the database, it will let us know if any operations failed.

### Unordered Bulk Write

- Has to be specified with the flag: `{ ordered: false }`
- Executes writes in parallel

If the writes in our batch don't have any causal relationship, then the client can send them over in an "Unordered Bulk Write". This will execute the write operations in parallel, so the writes are non-blocking. And as a result, a single failure won't prevent any of the other writes from succeeding. Now those writes might fail on their own, but their execution is not tied to the success of any other writes.

## Summary

- Bulk writes allow database clients to send multiple writes
- Can either be ordered or unordered

So in conclusion, bulk writes make it more efficient to insert many documents into the database, by sending them all at the same time.

These bulk writes can be ordered, which means writes are executed in the order they were sent to the database, and any errors will prevent subsequent writes from executing.

They can also be unordered, which means writes are executed in parallel, and errors don't affect the execution of other writes.

One small thing to note: in a sharded collection, ordered bulk writes are expected to take longer because different write operations need to be routed to their designated shards. An ordered bulk write might reach the mongos in one batch, but then it has to be serialized across the different shards. Regardless of their designated shard, the write operation needs to be evaluated to see if we should continue or exit the execution of the remainder of the batch.